<a href="https://colab.research.google.com/github/elebon26/DN9-467-final-project/blob/main/notebooks/Final_Ian_Hedges_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ian Hedges - DIVE Analysis

This notebook documents my individual contribution for the MGMT 467 Term Project, including:
- Prompt logs used during development
- A DIVE analysis for one substantive question
- An interactive Plotly figure based on project data
- A link to the Looker Studio dashboard



In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import plotly.express as px
import plotly.graph_objects as go


## Dashboard Section Link & Influence

**Looker Studio Dashboard:**  
https://lookerstudio.google.com/u/0/reporting/7583f6ff-725b-4507-b47e-4dd24c4c485d/page/ncciF

**Sections Developed:**  
- Temp Volatility vs Hourly Average
- Hourly change time-series visualization
- Finer formatting points like placement and arrangement of visualizations for reader ease


## Prompt Logs

Below are the primary prompts used throughout development and debugging.  
(Each entry includes: Prompt → Tool/Context → Output Summary → Action Taken)

---

### Prompt Log 1 — Evaluating KPI feasibility in Looker

**Prompt:**  
"Is it possible for Looker Studio to calculate hour‑over‑hour temperature changes natively, using something like LAG or other window-style logic?"

**Tool/Context:**  
ChatGPT + Looker Studio calculated fields

**Output Summary:**  
Looker Studio’s calculated fields cannot perform SQL-style window functions (such as LAG) on BigQuery data, so the transformation cannot be done directly in the dashboard layer.

**Action Taken:**  
Shifted the hour‑over‑hour delta computation into BigQuery by creating a precomputed table/view, then connected that dataset to Looker.

---

### Prompt Log 2 — Designing the BigQuery pattern for temperature deltas

**Prompt:**  
"What’s the most reliable BigQuery SQL pattern for generating hourly average temperatures and a 1‑hour difference metric for each city?"

**Tool/Context:**  
ChatGPT + BigQuery SQL

**Output Summary:**  
Recommended using hourly bucketing via `TIMESTAMP_TRUNC`, aggregating with `AVG()` by city and hour, and then applying `LAG()` over the hourly sequence to compute the delta.

**Action Taken:**  
Built a table similar to `live_weather_with_delta` containing:  
- city  
- ts_hour  
- temp_hour_avg  
- temp_delta_1h  

---

### Prompt Log 3 — Understanding Looker dashboard aggregation behavior

**Prompt:**  
"When multiple cities are selected in the filter, why does the time‑series chart appear to show an averaged line?"

**Tool/Context:**  
ChatGPT + Looker Studio chart configuration

**Output Summary:**  
A time‑series metric aggregated with AVG will combine values across all selected cities unless the chart is broken down by city or the filter is restricted to a single selection.

**Action Taken:**  
Kept multi‑select enabled for the table but clarified expected chart behavior. Depending on the design choice, used a city breakdown for the line chart or limited the KPI chart to single‑select.



# **DIVE Entry — Weather Anomaly Detection Using Open‑Meteo Data**

## **D — Define the Question**

**Question:**  
How can we automatically detect and categorize rapid weather anomalies like sudden temperature swings or abrupt wind‑speed changes using Open‑Meteo’s near‑real‑time data stream?

**Why it matters (business relevance):**  
Rapid weather anomalies often precede meaningful operational impacts: travel delays, safety concerns, event disruptions, or sudden environmental risks. Detecting these anomalies early provides users with actionable insights rather than raw data, improving decision‑making and situational awareness.

---

## **I — Investigate (Data + Method)**

**Data used:**  
- Streaming Open‑Meteo feed ingested into `live_weather`  
  - temperature  
  - windspeed  
  - timestamp  
  - city  
- Derived table: `weather_anomalies`  
  - hourly averages  
  - deltas  
  - anomaly flags  

**Approach:**  
1. **Bucket data into hourly intervals** to smooth minute‑level noise.  
2. **Compute deltas** for temperature and windspeed using `LAG()` within each city.  
3. **Define anomaly thresholds**, for example:  
   - Temperature spike: `delta_temp > 3.0`  
   - Temperature drop: `delta_temp < -3.0`  
   - Wind jump: `delta_wind > 5.0`  
4. **Assign anomaly categories**, such as:  
   - *Rapid Warming*  
   - *Rapid Cooling*  
   - *Wind Surge*  
   - *Normal*  
5. **Store results in a precomputed BigQuery table** for fast dashboard rendering.  
6. **Visualize anomalies in Looker Studio** using:  
   - A time‑series chart with color‑coded anomaly markers  
   - A table listing the latest anomalies per city  
   - Optional KPI tiles (e.g., “Cities with active anomalies”)  

---

## **V — Validate (Proof it works)**

**Validation steps:**  
- Verified that hourly buckets align with Open‑Meteo timestamps.  
- Manually checked several known weather events to confirm anomaly detection triggers.  
- Ensured thresholds produce meaningful, not overly noisy, classifications.  
- Confirmed Looker Studio updates anomaly flags as new streaming data arrives.  
- Cross‑checked delta calculations against raw Open‑Meteo values for accuracy.

---

## **E — Explain (Conclusion + Next Step)**

**Conclusion:**  
By precomputing deltas and anomaly categories in BigQuery, we can reliably detect rapid weather changes that Looker Studio cannot compute natively. This approach transforms raw Open‑Meteo data into actionable intelligence, enabling a dashboard that highlights meaningful events rather than just displaying numbers.

**Next improvement:**  
Introduce a **severity score** (e.g., mild, moderate, severe) based on the magnitude of the anomaly, and use conditional formatting or icons in Looker Studio to make high‑impact events stand out instantly.



In [3]:
from google.colab import auth
auth.authenticate_user()

In [9]:
from google.cloud import bigquery
import pandas as pd

# Initialize client with explicit project ID
project_id = 'finalprojectfor467' # Extracted from the dataset path in the query
client = bigquery.Client(project=project_id)

# SQL query for your table
query = """
SELECT
    city,
    ts_hour,
    temp_hour_avg,
    temp_delta_1h
FROM `finalprojectfor467.weather_proj.live_weather_with_delta`
ORDER BY city, ts_hour
"""

# Run query
df = client.query(query).to_dataframe()
df.head()

,city,ts_hour,temp_hour_avg,temp_delta_1h
0,Albuquerque,2025-12-11 22:00:00+00:00,19.611348,NaN
1,Albuquerque,2025-12-11 23:00:00+00:00,17.605686,-2.005662
2,Albuquerque,2025-12-12 00:00:00+00:00,14.533727,-3.071960
3,Albuquerque,2025-12-12 01:00:00+00:00,13.201835,-1.331892
4,Albuquerque,2025-12-12 02:00:00+00:00,12.048496,-1.153339


In [10]:
import plotly.express as px

fig = px.line(
    df,
    x="ts_hour",
    y="temp_delta_1h",
    color="city",
    title="Hour-over-Hour Temperature Change by City",
    labels={
        "ts_hour": "Timestamp (Hourly)",
        "temp_delta_1h": "Temperature Δ (°C)"
    }
)

fig.update_layout(template="plotly_white")
fig.show()

In [11]:
df["anomaly_flag"] = df["temp_delta_1h"].abs() > 3

fig2 = px.scatter(
    df,
    x="ts_hour",
    y="temp_delta_1h",
    color="anomaly_flag",
    color_discrete_map={True: "red", False: "gray"},
    title="Temperature Anomalies (|Δ| > 3°C)",
    labels={"anomaly_flag": "Anomaly"}
)

fig2.update_layout(template="plotly_white")
fig2.show()

## Summary of My Contribution

- Assisted in ensuring live streaming data showed properly in Looker and in Python testing.
- Produced an interactive Plotly figure to show the delta logic and also identify potential anomalies to help understand the data.

